In [1]:
from pyspark.sql import SparkSession

#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-CSET+domains+sample") \
    .config("spark.executor.memory","28g") \
    .config("spark.driver.memory","28g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-CSET+domains+sample").getOrCreate()
spark.conf.set("spark.worker.cleanup.enabled",True)
spark.conf.set("spark.worker.cleanup.interval",1800)
spark.conf.set("spark.worker.cleanup.appDataTtl",3600)
spark.conf.set("spark.sql.shuffle.partitions",1000)

#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/13 13:49:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet('s3a://ter-2024/ddomain-240209/hashdom6-20/part-00975-2145dc48-c2f0-4de4-8c33-8b1231e3e36a-c000.gz.parquet')

#df = spark.read.parquet('s3a://ter-2024/ddomain-240209/hashdom6-20/')

#df.printSchema()

df.show()

print("Nombre de lignes:", df.count())



24/02/13 13:49:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+-----+--------------------+----------------------+
|                pset|count|              sample|distinct_domains_count|
+--------------------+-----+--------------------+----------------------+
|<schema.org/descr...|   97|https://www.gotah...|                     4|
|<schema.org/conta...|    3|https://www.leadc...|                     3|
|<schema.org/addre...| 2683|http://chelyabins...|                    40|
|<schema.org/autho...|    5|http://rodrigo-si...|                     3|
|isa:<welcome.vara...|    1|https://welcome.v...|                     1|
|<schema.org/agent...|   63|https://www.delaw...|                     1|
|<schema.org/autho...|  697|https://mgronline...|                    13|
|<schema.org/autho...|   44|https://dinnerton...|                     2|
|isa:<welcome.vara...|    3|https://welcome.v...|                     1|
|<schema.org/addit...|    1|https://myphamhuy...|                     1|
|<schema.org/descr...|    2|https://groupes.l...|  

Nombre de lignes: 57
